GRUPO: 
Caio Basso,
Pedro Miczewski,
Rafael Lopes,
Vitor Emmanuel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab.patches import cv2_imshow
from glob import glob

import cv2

import itertools    

import re 

import datetime

import time

from IPython import display

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.models import Model

Para o load do dataset, primeiramente excluimos imagens que poderiam afetar no desempenho do modelo, como por exemplo imagens do Homer em que só apareciam os olhos ou mais de um personagem na mesma imagem.

In [ ]:
!mkdir /content/Simpsons
!mkdir /content/Simpsons/Treino
!mkdir /content/Simpsons/Teste

In [ ]:
!wget https://github.com/rafz1/TDE_Simpsons/blob/main/Simpsons_old.zip?raw=true -O Simpsons_old.zip

--2022-11-14 19:16:17--  https://github.com/rafz1/TDE_Simpsons/blob/main/Simpsons_old.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rafz1/TDE_Simpsons/raw/main/Simpsons_old.zip [following]
--2022-11-14 19:16:17--  https://github.com/rafz1/TDE_Simpsons/raw/main/Simpsons_old.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rafz1/TDE_Simpsons/main/Simpsons_old.zip [following]
--2022-11-14 19:16:17--  https://raw.githubusercontent.com/rafz1/TDE_Simpsons/main/Simpsons_old.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
!unzip -o Simpsons_old.zip -d '/content/'

Archive:  Simpsons_old.zip
   creating: /content/Simpsons_old/
   creating: /content/Simpsons_old/Teste/
  inflating: /content/Simpsons_old/Teste/bart082.jpg  
  inflating: /content/Simpsons_old/Teste/bart083.jpg  
  inflating: /content/Simpsons_old/Teste/bart084.jpg  
  inflating: /content/Simpsons_old/Teste/bart085.jpg  
  inflating: /content/Simpsons_old/Teste/bart090.jpg  
  inflating: /content/Simpsons_old/Teste/bart099.jpg  
  inflating: /content/Simpsons_old/Teste/bart103.jpg  
  inflating: /content/Simpsons_old/Teste/bart110.jpg  
  inflating: /content/Simpsons_old/Teste/bart112.jpg  
  inflating: /content/Simpsons_old/Teste/bart113.jpg  
  inflating: /content/Simpsons_old/Teste/bart114.jpg  
  inflating: /content/Simpsons_old/Teste/bart115.jpg  
  inflating: /content/Simpsons_old/Teste/homer077.jpg  
  inflating: /content/Simpsons_old/Teste/homer078.jpg  
  inflating: /content/Simpsons_old/Teste/homer080.jpg  
  inflating: /content/Simpsons_old/Teste/homer082.jpg  
  inflating

In [ ]:
bart_train = glob(os.path.join('/content/Simpsons_old/Treino', 'bart*'))
bart_test = glob(os.path.join('/content/Simpsons_old/Teste/', 'bart*'))

homer_train = glob(os.path.join('/content/Simpsons_old/Treino/', 'homer*'))
homer_test = glob(os.path.join('/content/Simpsons_old/Teste/', 'homer*'))

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 256, 256

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # Resizing to 286x286
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # Random cropping back to 256x256
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # Random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

In [ ]:
def load(image_file, label_file):
  # Read and decode an image file to a uint8 tensor
  # image = tf.io.read_file(image_file)
  # image = tf.io.decode_bmp(image)

  # homer = tf.io.read_file(label_file)
  # homer = tf.io.decode_bmp(homer)

  # Convert both images to float32 tensors
  input_image = tf.cast(image_file, tf.float32)
  homer_image = tf.cast(label_file, tf.float32)


  return input_image, homer_image

In [ ]:
def load_image_train(image_file, label_file):
  input_image, real_image = load(image_file, label_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
input_shape_ = (256, 256, 3)

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    #layers.RandomZoom(0.1),
  ]
)

In [ ]:
simpsons_dir = os.path.join("Simpsons")
treino_dir = os.path.join(simpsons_dir, "Treino")
teste_dir = os.path.join(simpsons_dir, "Teste")

In [ ]:
i = 0

for img in bart_train:
    bart = cv2.imread(img)
    bart = cv2.resize(bart, (256, 256))

    for img_homer in homer_train:
      homer = cv2.imread(img_homer)
      homer = cv2.resize(homer, (256, 256))
      
      augmented_bart = data_augmentation(bart)
      augmented_homer = data_augmentation(homer)

      bart = np.array(augmented_bart)
      homer = np.array(augmented_homer)

      #bart, homer = load_image_train(bart, homer)

      cv2.imwrite(os.path.join(treino_dir, "bart" + str(i)) + ".jpg", \
        bart)
      
      cv2.imwrite(os.path.join(treino_dir, "homer" + str(i)) + ".jpg", \
        homer)
      
      i += 1

In [ ]:
i = 0

for img in bart_test:
    bart = cv2.imread(img)
    bart = cv2.resize(bart, (256, 256))

    for img_homer in homer_test:
      homer = cv2.imread(img_homer)
      homer = cv2.resize(homer, (256, 256))
      
      augmented_bart = data_augmentation(bart)
      augmented_homer = data_augmentation(homer)

      bart = np.array(augmented_bart)
      homer = np.array(augmented_homer)

      #bart, homer = load_image_train(bart, homer)

      cv2.imwrite(os.path.join(teste_dir, "bart" + str(i)) + ".jpg", \
        bart)
      
      cv2.imwrite(os.path.join(teste_dir, "homer" + str(i)) + ".jpg", \
        homer)
      
      i += 1

Todas as imagens foram redimensionadas para 256x256. A partir de cada imagem do bart, foi carregado uma imagem do homer e aplicado data augmentation.

In [ ]:
len_dataset = glob(os.path.join('/content/Simpsons/Treino/', 'bart*'))

In [ ]:
len([name for name in len_dataset if os.path.isfile(name)])

480

In [ ]:
len_test_dataset = glob(os.path.join('/content/Simpsons/Teste/', 'bart*'))

In [ ]:
len([name for name in len_test_dataset if os.path.isfile(name)])

72

In [ ]:
!zip -r /content/Simpsons.zip /content/Simpsons

  adding: content/Simpsons/ (stored 0%)
  adding: content/Simpsons/Treino/ (stored 0%)
  adding: content/Simpsons/Treino/bart291.jpg (deflated 2%)
  adding: content/Simpsons/Treino/homer377.jpg (deflated 2%)
  adding: content/Simpsons/Treino/bart330.jpg (deflated 2%)
  adding: content/Simpsons/Treino/homer221.jpg (deflated 3%)
  adding: content/Simpsons/Treino/bart24.jpg (deflated 2%)
  adding: content/Simpsons/Treino/bart116.jpg (deflated 2%)
  adding: content/Simpsons/Treino/homer290.jpg (deflated 3%)
  adding: content/Simpsons/Treino/homer64.jpg (deflated 2%)
  adding: content/Simpsons/Treino/bart419.jpg (deflated 2%)
  adding: content/Simpsons/Treino/bart109.jpg (deflated 2%)
  adding: content/Simpsons/Treino/homer203.jpg (deflated 3%)
  adding: content/Simpsons/Treino/homer373.jpg (deflated 3%)
  adding: content/Simpsons/Treino/homer288.jpg (deflated 3%)
  adding: content/Simpsons/Treino/bart185.jpg (deflated 3%)
  adding: content/Simpsons/Treino/bart313.jpg (deflated 2%)
  adding

In [ ]:
from google.colab import files
files.download("/content/Simpsons.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>